In [ ]:
import pickle
import os
from utils import get_kmers, check_response, emit_metrics
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score

min_mer, max_mer = 8, 13
testset = {
        "vh": 
            {
            "human": "test_human_VH_shuf_no_dupli_aligned_10k.txt",      
            "diverse": "diverse_5_human_VH_biophi_and_test_seqs.txt",       
            "mouse": "mouse_VH_cleaned_aligned_10k.txt",      
            "rhesus": "rhesus_VH_cleaned_aligned_10k.txt",      
            "pssm": "rd_pssm_human_VH_random_10k.txt",
            },
        "vkappa":  
            {
            "human": "test_human_VKappa_shuf_no_dupli_aligned_10k.txt",  
            "diverse": "diverse_2_5_human_VKappa_biophi_and_test_seqs.txt", 
            "mouse": "mouse_VKappa_cleaned_aligned_10k.txt",  
            "rhesus": "rhesus_VKappa_cleaned_aligned_10k.txt",  
            "pssm": "rd_pssm_human_VKappa_random_10k.txt",
            },
        "vlambda": 
            {
            "human": "test_human_VLambda_shuf_no_dupli_aligned_10k.txt", 
            "diverse": "diverse_2_5_VLambda_biophi_and_test_seqs.txt",      
            "mouse": "mouse_VLambda_cleaned_aligned_all.txt", 
            "rhesus": "rhesus_VLambda_cleaned_aligned_10k.txt", 
            "pssm": "rd_pssm_human_VLambda_random_10k.txt",
            },
        }

def read_txt(fname):
    seqs = []
    if os.path.exists(fname):
        with open(fname, 'r') as f:
            lines = f.readlines()
            for line in lines:
                seqs.append(line.strip().replace('-', ''))
        return seqs

def calc_scores(sequences, pool_pos=[], pool_neg=[], label=0):
    pred, true = [], []
    assert len(sequences) > 0
    for i in range(len(sequences)):
        scores, response, total_kmers = {}, 0, 0
        for mer in range(min_mer, max_mer):
            kmers = get_kmers(sequences[i], mer, mer)
            pool_mer_pos = []
            for item in pool_pos:
                if mer not in item.keys(): continue
                pool_mer_pos.append(item[mer])
            pool_mer_neg = []
            for item in pool_neg:
                if mer not in item.keys(): continue
                pool_mer_neg.append(item[mer])
            subscores, subresponse = check_response(kmers, mer, pool_mer_pos, pool_mer_neg, scores)
            response += subresponse
            total_kmers += len(kmers)
            scores = subscores
        pred.append(response / total_kmers)
        true.append(label)
    return pred, true

def classification(pred, label, nfloat=5):
    intervals = np.arange(0, 1, 0.0001)
    auroc = roc_auc_score(label, pred)
    auprc = average_precision_score(label, pred)
    print('interval,mcc,auprc,auroc,accuracy,f1_score,recall,precision')
    for interval in intervals:
        fpred = [1 if i >= interval else 0 for i in pred]
        metric = emit_metrics(fpred, label)
        metric = map(lambda x: round(x, nfloat), metric)
        accuracy, f1_score, mcc, recall, precision = metric
        print(f"{round(interval, nfloat)},{mcc},{auprc},{auroc},{accuracy},{f1_score},{recall},{precision}")
        
data_human, data_oas_pair_human, data_oas_pair_mouse = {}, {}, {}
for mer in range(min_mer, max_mer):
    if mer not in data_oas_pair_human.keys():
        pkl = f'data/oas_paired_human_{str(mer)}mer.dump'
        with open(pkl, 'rb') as f:
            pools = pickle.load(f)
        data_oas_pair_human[mer] = pools
    if mer not in data_human.keys():
        pkl = f'data/human_{str(mer)}mer.dump'
        with open(pkl, 'rb') as f:
            pools = pickle.load(f)
        data_human[mer] = pools
    if mer not in data_oas_pair_mouse.keys():
        pkl = f'data/oas_paired_mouse_{str(mer)}mer.dump'
        with open(pkl, 'rb') as f:
            pools = pickle.load(f)
        data_oas_pair_mouse[mer] = pools

In [ ]:
pool_pos = [data_human, data_oas_pair_human]
pool_neg = [data_oas_pair_mouse]
abnativ_dataset = 'vkappa' # choose vh, vlambda, vkappa
diverse_seqs = read_txt(f"data/abnativ/{abnativ_dataset}/{testset[abnativ_dataset]['diverse']}")
mouse_seqs = read_txt(f"data/abnativ/{abnativ_dataset}/{testset[abnativ_dataset]['mouse']}")
rhesus_seqs = read_txt(f"data/abnativ/{abnativ_dataset}/{testset[abnativ_dataset]['rhesus']}")
pssm_seqs = read_txt(f"data/abnativ/{abnativ_dataset}/{testset[abnativ_dataset]['pssm']}")
human_seqs = read_txt(f"data/abnativ/{abnativ_dataset}/{testset[abnativ_dataset]['human']}")
tasks = {}
diverse_pred, diverse_true = calc_scores(diverse_seqs, pool_pos=pool_pos, pool_neg=pool_neg, label=1)
mouse_pred, mouse_true = calc_scores(mouse_seqs,   pool_pos=pool_pos, pool_neg=pool_neg, label=0)
rhesus_pred, rhesus_true = calc_scores(rhesus_seqs,  pool_pos=pool_pos, pool_neg=pool_neg, label=0)
pssm_pred, pssm_true = calc_scores(pssm_seqs,    pool_pos=pool_pos, pool_neg=pool_neg, label=0)
human_pred, human_true = calc_scores(human_seqs,   pool_pos=pool_pos, pool_neg=pool_neg, label=1)
tasks['human_mouse'] = {'pred': human_pred + mouse_pred, 'true': human_true + mouse_true}
tasks['human_pssm'] = {'pred': human_pred + pssm_pred, 'true': human_true + pssm_true}
tasks['human_rhesus'] = {'pred': human_pred + rhesus_pred, 'true': human_true + rhesus_true}
tasks['diverse_human_mouse'] = {'pred': diverse_pred + mouse_pred, 'true': diverse_true + mouse_true}
tasks['diverse_human_pssm'] = {'pred': diverse_pred + pssm_pred, 'true': diverse_true + pssm_true}
tasks['diverse_human_rhesus'] = {'pred': diverse_pred + rhesus_pred, 'true': diverse_true + rhesus_true}    

In [ ]:
for k, v in tasks.items():
    task_name, pred, true = k, v['pred'], v['true']
    print(f"{abnativ_dataset} -- {task_name}")
    print("############################")    
    classification(pred, true)
    print("############################")

vkappa -- human_mouse
############################
0.9924602285228522 0.9892222963332864
############################
